In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import sys, subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-U",
    "langchain-community",
    "langchain-upstage",
    "langchain-text-splitters",
    "faiss-cpu",
    "pypdf"
])

In [8]:
def main():
    load_dotenv()
    pdf_path = "c:/rookies4dev/mylangchain/mylangchain-app/src/hw/hw_data/콘텐츠분쟁해결_사례.pdf"

    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        separators=[
            "\n【사건개요】",
            "\n【쟁점사항】",
            "\n【처리경위】",
            "\n【처리결과】",
            "\n■", "\n\n", "\n", ".", " ", ""
        ]
    )
    splits = text_splitter.split_documents(docs)

    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.2
    )

    prompt_tmpl = (
        "당신은 콘텐츠 분야 전문 법률 자문가입니다.\n"
        "아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공하세요.\n\n"
        "관련 분쟁사례:\n{context}\n\n"
        "상담 내용: {question}\n\n"
        "답변 가이드라인:\n"
        "1. 제시된 사례들을 근거로 답변하세요\n"
        "2. 관련 법령이나 조항이 있다면 명시하세요\n"
        "3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요\n"
        "4. 실무적 해결방안을 단계별로 제시하세요\n"
        "5. 사례에 없는 내용은 \"제시된 사례집에서는 확인할 수 없습니다\"라고 명시하세요\n\n"
        "전문 법률 조언:"
    )
    prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_tmpl)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    test_questions = [
        "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
        "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
        "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
        "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
        "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
    ]

    def classify_dispute_type(query: str) -> str:
        game = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
        elearn = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
        web = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
        q = query.lower()
        if any(k in q for k in game):
            return "게임"
        if any(k in q for k in elearn):
            return "이러닝"
        if any(k in q for k in web):
            return "웹콘텐츠"
        return "기타"

    for q in test_questions:
        r = qa_chain.invoke(q)
        print("질문:", q)
        print("분류:", classify_dispute_type(q))
        print("답변:", r["result"])
        if "source_documents" in r and r["source_documents"]:
            refs = []
            for d in r["source_documents"]:
                meta = d.metadata or {}
                page = meta.get("page", None)
                refs.append(f"p.{page+1}" if isinstance(page, int) else "p.?")
            print("참조:", ", ".join(refs))
        print("-" * 80)

if __name__ == "__main__":
    main()

질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분류: 게임
답변: ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사례 해결 방안  

#### 1. **사례 기반 법적 쟁점 분석**  
제시된 사례(2006, 2009년 한국소비자원 조정사례)에 따르면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 해결 여부가 결정됩니다:  
- **(1) 시스템 오류의 객관적 입증**:  
  - 2009년 사례(1,000억 게임머니 소실)에서는 신청인의 주장만으로는 시스템 오류를 인정하기 어려웠으며, 다른 이용자의 유사 피해 사례가 없어 복구 요청이 기각되었습니다.  
  - 반면 2006년 사례(프로그램 오류)에서는 게임사가 오류를 인정하고 복구를 진행했습니다.  
  → **해결 방안**: 게임 내 로그, 서버 기록, 다른 이용자의 피해 사례 등을 수집하여 시스템 오류를 객관적으로 입증해야 합니다.  

- **(2) 계정 소유권 및 약관 준수 여부**:  
  - 2006년 사례(계정 명의 불일치)에서는 게임사가 계정 공유 또는 현금거래를 금지하는 약관을 근거로 복구를 거부했습니다.  
  → **해결 방안**: 계정 명의가 본인인지 확인하고, 게임사의 이용약관에서 "계정 공유 금지" 조항이 있는지 검토해야 합니다.  

- **(3) 게임사의 책임 인정 여부**:  
  - 2007년 사례(해킹 아이템 회수)에서는 게임사가 저작권 및 이용약관을 근거로 아이템 회수를 정당화했습니다.  
  → **해결 방안**: 시스템 오류가 아닌 해킹 등 제3자 행위로 인한 손실일 경우, 게임사의 책임이 제한될 수 있음을 유의해야 합니다.  

---

#### 2. **관련 법령 및 약관 검토**  
- **민법 제250조(도품·유실물 특례)**:  
  - 아이템이 "금전"에 준하는 가상재화인 경우, 반환 청구가 어려울 수 있습니다. 다만, 게임사가 오류를